<a href="https://colab.research.google.com/github/JEETMANISHBOBDE/Image-Restoration/blob/main/Image_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import io
import os

# Upload file(s)
uploaded = files.upload()

# Create a directory to store the data
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Process each uploaded file
for file_name in uploaded.keys():
    print(f'Uploaded file: {file_name}')
    if file_name.endswith('.zip'):
        # If the file is a zip file, extract its contents
        with zipfile.ZipFile(io.BytesIO(uploaded[file_name]), 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print(f'Extracted {file_name} into the "{data_dir}" folder.')
    else:
        # If it's not a zip file, save it directly into the data directory
        with open(os.path.join(data_dir, file_name), 'wb') as f:
            f.write(uploaded[file_name])
        print(f'Saved {file_name} into the "{data_dir}" folder.')
